In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
test = torch.randn([3, 8, 60])

In [69]:
conv_1 = nn.Conv2d(3, 64, (1, 3), padding=0)
conv_2 = nn.Conv2d(64, 32, (1, 7), padding=0)
conv_3 = nn.Conv2d(32, 16, (1, 10), padding=0)
conv_4 = nn.Conv2d(16, 3, (1, 14), padding=0)

In [72]:
test = conv_4(conv_3(conv_2(conv_1(test))))

In [71]:
convt_1 = nn.ConvTranspose2d(3, 16, (1, 14), padding=0)
convt_2 = nn.ConvTranspose2d(16, 32, (1, 10), padding=0)
convt_3 = nn.ConvTranspose2d(32, 64, (1, 7), padding=0)
convt_4 = nn.ConvTranspose2d(64, 3, (1, 3), padding=0)

In [75]:
convt_4(convt_3(convt_2(convt_1(test)))).shape

torch.Size([3, 8, 60])

In [43]:
pool_1 = nn.MaxPool2d((1, 2), (1, 2))
pool_1(conv_1(test)).shape

torch.Size([16, 10, 30])

In [2]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/Close.csv', index_col=0, parse_dates=True)

In [3]:
np.array((np.log(df) - np.log(df.shift(1))).dropna()).shape

(814, 8)

In [4]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X = list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x = sequence[i:end_ix]
		X.append(seq_x)
	return np.array(X)

In [22]:
test = split_sequence(np.array((np.log(df) - np.log(df.shift(1))).dropna()), 60)

In [15]:
test.shape

(754, 60, 8)

In [6]:
a = np.expand_dims(np.transpose(test, (0, 2, 1)), axis=1)
b = np.expand_dims(np.transpose(test, (0, 2, 1)), axis=1)

In [12]:
c = np.concatenate([a, b], axis=1)

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [20]:
for i in range(test.shape[0]):
    scaler = StandardScaler()
    test[i, :, :] = scaler.fit_transform(test[i, :, :]) 

In [23]:
test

array([[[-7.34918596e-03, -2.87277383e-03, -1.46036550e-03, ...,
         -1.28618272e-03,  4.62901264e-03, -1.76423049e-03],
        [-8.35545832e-03, -3.72118520e-03, -3.21757156e-02, ...,
         -3.28278129e-02,  2.73737876e-03, -3.23662102e-02],
        [-1.67667340e-01, -5.90079885e-02, -2.22722651e-02, ...,
         -2.12731054e-02,  1.58559969e-04, -2.15537694e-02],
        ...,
        [ 1.45717371e-01,  4.30708228e-02,  1.03979094e-03, ...,
          7.82917896e-04,  1.58056160e-03,  7.83323601e-04],
        [-3.85483340e-02, -2.54692506e-02, -2.41520683e-02, ...,
         -2.41600024e-02, -4.52418154e-03, -2.34466639e-02],
        [ 3.86950338e-02,  8.74587229e-03,  3.29470389e-02, ...,
          3.24735996e-02, -5.35876033e-03,  3.25647533e-02]],

       [[-8.35545832e-03, -3.72118520e-03, -3.21757156e-02, ...,
         -3.28278129e-02,  2.73737876e-03, -3.23662102e-02],
        [-1.67667340e-01, -5.90079885e-02, -2.22722651e-02, ...,
         -2.12731054e-02,  1.58559969e

In [24]:
test.shape

(754, 60, 8)

In [24]:
from environment.env import TradingEnv
import numpy as np

In [16]:
env = TradingEnv('data', observation_features='All')
for episode in range(1):
    s0 = env.reset()
    episode_reward = 0
    done = False
    while True:
        s1, r1, done, info = env.step(np.random.randn(8))
        episode_reward += r1
        s0 = s1
        if done: 
            break
    
    print(info)

{'reward': -18.452054307996516, 'log_return': -18.452054307996516, 'portfolio_value': 0.0, 'return': -1.0689422236752235, 'rate_of_return': -1.0, 'weights_mean': 0.125, 'weights_std': 0.12789006347965567, 'cost': 0.015589408894973787, 'date': numpy.datetime64('2019-11-06T00:00:00.000000000'), 'steps': 4, 'market_value': 0.0588405728098063}


In [3]:
s0 = env.reset()
env.step(np.random.randn(8))

({'portfolio': array([[[-0.00907256,  0.04795925, -0.04296543, ..., -0.0126848 ,
           -0.00320305,  0.02161533],
          [-0.00776201,  0.1126946 , -0.02973906, ..., -0.03498391,
           -0.00591802, -0.00350815],
          [ 0.00123718, -0.00208921, -0.00515751, ..., -0.00340304,
           -0.0018004 ,  0.00724175],
          ...,
          [-0.00504076, -0.0017592 ,  0.00490357, ...,  0.00452074,
           -0.0084932 ,  0.00687094],
          [-0.00565114, -0.00106827,  0.00463469, ...,  0.00475945,
           -0.01246205,  0.00702652],
          [-0.00695566,  0.00261404,  0.00526948, ...,  0.00300026,
           -0.0104822 ,  0.00667265]],
  
         [[-0.00649135,  0.01800543,  0.00612152, ..., -0.00340235,
           -0.03730407,  0.00318588],
          [-0.01067782,  0.06514885,  0.01106969, ..., -0.02125103,
           -0.01002629, -0.02634136],
          [ 0.00398417,  0.0020325 , -0.00166137, ..., -0.00915629,
            0.00842609,  0.00047676],
          ...,

In [23]:
w1 = np.clip(np.random.randn(8), a_min=0, a_max=1)
# w1 = np.insert(w1, 0, np.clip(1 - w1.sum(), a_min=0, a_max=1))
w1 = w1 / w1.sum()
w1

array([0.17540859, 0.17540859, 0.        , 0.17540859, 0.17540859,
       0.12295705, 0.17540859, 0.        ])

In [19]:

w1 = np.insert(w1, 0, np.clip(1 - w1.sum(), a_min=0, a_max=1))
w1 = w1 / w1.sum()
w1

array([0.        , 0.39726231, 0.        , 0.08049415, 0.30070609,
       0.02861364, 0.        , 0.        , 0.19292381])

In [35]:
env.action_space.high.shape[0]

8